In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 3.6 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
     |████████████████████████████████| 6.6 MB 29.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import os
import string
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from transformers import TFBertModel, BertTokenizer,BertConfig
#build model
from tensorflow.keras.layers import Input, Dropout, Dense,LSTM
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import re
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
label_encoder=preprocessing.LabelEncoder()


In [ ]:
!gdown --id 1JGYt2F1Aqhm3YPXNVgZe-hzrPO4AMjKt
df=pd.read_csv('/content/P22_Database_original.csv',sep=';', encoding='ISO-8859-1')

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1JGYt2F1Aqhm3YPXNVgZe-hzrPO4AMjKt
To: /content/P22_Database_original.csv
100% 82.5M/82.5M [00:00<00:00, 252MB/s]


In [ ]:

df2=df
df

,Unnamed: 0,ID,Text,O,C,E,A,N
0,1,74821190,mentirosos compulsivos es simplista y feliz so...,2,2,1,2,1.0
1,2,233401415,alguien que te da una razón para no confiar en...,1,2,2,1,1.0
2,3,211340264,"los políticos es lo que haces, no tiene que es...",1,0,0,0,2.0
3,4,149202456,Alguien que te apuñala por la espalda. Tiene u...,2,2,1,1,2.0
4,5,236017626,ALGUIEN QUE SE ENCUENTRA DESCONFIADO EN SUS BE...,0,1,0,1,0.0
...,...,...,...,...,...,...,...,...
12463,12464,f83aa7290cf3ae8ed75d6aee5ebbb559,¡Jï¿½g ï¿½slkar sverige! D-4sweden! D-1D -6 ha...,0,1,0,1,0.0
12464,12465,fbe5aa478508d1dc931427ade5d9e1b4,"Totalizó mi auto anoche. Por suerte, nadie res...",1,2,1,0,1.0
12465,12466,fc1c9fb6c64740edcbbf8cfe9dde8b02,"reglas de youtube. Honestamente, el día verde ...",1,0,1,1,1.0
12466,12467,fe22087986fdcc65939c793fe0ec90a9,me levanté temprano hoy para un viaje a Hambur...,0,1,1,0,1.0


In [ ]:
df2

,Unnamed: 0,ID,Text,O,C,E,A,N
0,1,74821190,mentirosos compulsivos es simplista y feliz so...,2,2,1,2,1.0
1,2,233401415,alguien que te da una razón para no confiar en...,1,2,2,1,1.0
2,3,211340264,"los políticos es lo que haces, no tiene que es...",1,0,0,0,2.0
3,4,149202456,Alguien que te apuñala por la espalda. Tiene u...,2,2,1,1,2.0
4,5,236017626,ALGUIEN QUE SE ENCUENTRA DESCONFIADO EN SUS BE...,0,1,0,1,0.0
...,...,...,...,...,...,...,...,...
12463,12464,f83aa7290cf3ae8ed75d6aee5ebbb559,¡Jï¿½g ï¿½slkar sverige! D-4sweden! D-1D -6 ha...,0,1,0,1,0.0
12464,12465,fbe5aa478508d1dc931427ade5d9e1b4,"Totalizó mi auto anoche. Por suerte, nadie res...",1,2,1,0,1.0
12465,12466,fc1c9fb6c64740edcbbf8cfe9dde8b02,"reglas de youtube. Honestamente, el día verde ...",1,0,1,1,1.0
12466,12467,fe22087986fdcc65939c793fe0ec90a9,me levanté temprano hoy para un viaje a Hambur...,0,1,1,0,1.0


In [ ]:
def text_cleaning(t):
  a=t
  a=a.lower()
  x=re.sub('\d',' ',a)
  x.split()
  ' '.join(x)
  new_string = x.translate(str.maketrans('', '', string.punctuation))


  return new_string

In [ ]:
df2.isnull().sum()

Unnamed: 0       0
ID               0
Text            54
O                0
C                0
E                0
A                0
N             8677
dtype: int64

In [ ]:
df2.dropna(axis=0,inplace=True)
df2.isnull().sum()

Unnamed: 0    0
ID            0
Text          0
O             0
C             0
E             0
A             0
N             0
dtype: int64

In [ ]:
df2

,Unnamed: 0,ID,Text,O,C,E,A,N
0,1,74821190,mentirosos compulsivos es simplista y feliz so...,2,2,1,2,1.0
1,2,233401415,alguien que te da una razón para no confiar en...,1,2,2,1,1.0
2,3,211340264,"los políticos es lo que haces, no tiene que es...",1,0,0,0,2.0
3,4,149202456,Alguien que te apuñala por la espalda. Tiene u...,2,2,1,1,2.0
4,5,236017626,ALGUIEN QUE SE ENCUENTRA DESCONFIADO EN SUS BE...,0,1,0,1,0.0
...,...,...,...,...,...,...,...,...
12463,12464,f83aa7290cf3ae8ed75d6aee5ebbb559,¡Jï¿½g ï¿½slkar sverige! D-4sweden! D-1D -6 ha...,0,1,0,1,0.0
12464,12465,fbe5aa478508d1dc931427ade5d9e1b4,"Totalizó mi auto anoche. Por suerte, nadie res...",1,2,1,0,1.0
12465,12466,fc1c9fb6c64740edcbbf8cfe9dde8b02,"reglas de youtube. Honestamente, el día verde ...",1,0,1,1,1.0
12466,12467,fe22087986fdcc65939c793fe0ec90a9,me levanté temprano hoy para un viaje a Hambur...,0,1,1,0,1.0


In [ ]:
df2['Text']=df2['Text'].apply(text_cleaning)

In [ ]:
df2['A'].value_counts()

2    1672
0    1516
1     549
Name: A, dtype: int64

In [ ]:
X1=df2['Text']
X1

0        mentirosos compulsivos es simplista y feliz so...
1        alguien que te da una razón para no confiar en...
2        los políticos es lo que haces no tiene que est...
3        alguien que te apuñala por la espalda tiene un...
4        alguien que se encuentra desconfiado en sus be...
                               ...                        
12463    ¡jï¿½g ï¿½slkar sverige d sweden d d   hasta s...
12464    totalizó mi auto anoche por suerte nadie resul...
12465    reglas de youtube honestamente el día verde su...
12466    me levanté temprano hoy para un viaje a hambur...
12467    está volando de regreso el viernes por la noch...
Name: Text, Length: 3737, dtype: object

In [ ]:
def buildmodel(X1,Y1):
  x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.1)
  x_tokenize=tokenizer(text=x_train.tolist(),padding=True, truncation=True,max_length=100,return_tensors='tf',verbose=True)
  X=x_tokenize['input_ids']
  Y=y_train
  model.fit(X,Y,batch_size=15,epochs=5,validation_split=0.2)
  x_tokenize=tokenizer(text=x_test.tolist(),padding=True, truncation=True,max_length=100,return_tensors='tf',verbose=True)
  print('\nEvaluatig Model--->\n')
  model.evaluate(x=x_tokenize['input_ids'],y=y_test)
  return model;

In [ ]:
df2['O']= label_encoder.fit_transform(df2['O'])
y_O=to_categorical(df2['O'],dtype='int32')
model_O=buildmodel(X1,y_O)
model.save(os.path.join('model_O.h5', '/content/drive/MyDrive/Models/model_O.h5'))

Epoch 1/5
 45/180 [======>.......................] - ETA: 36:05 - loss: 1.0152 - accuracy: 0.4444

In [ ]:
df2['C']= label_encoder.fit_transform(df2['C'])
y_C=to_categorical(df2['C'],dtype='int32')
model_C=buildmodel(X1,y_C)

Epoch 1/5
180/180 [==============================] - 83s 382ms/step - loss: 0.9606 - accuracy: 0.4323 - val_loss: 0.9263 - val_accuracy: 0.4636
Epoch 2/5
180/180 [==============================] - 66s 365ms/step - loss: 0.9286 - accuracy: 0.4349 - val_loss: 0.9239 - val_accuracy: 0.4636
Epoch 3/5
180/180 [==============================] - 65s 364ms/step - loss: 0.9273 - accuracy: 0.4513 - val_loss: 0.9209 - val_accuracy: 0.4636
Epoch 4/5
180/180 [==============================] - 66s 365ms/step - loss: 0.9290 - accuracy: 0.4450 - val_loss: 0.9190 - val_accuracy: 0.4636
Epoch 5/5
180/180 [==============================] - 66s 365ms/step - loss: 0.9282 - accuracy: 0.4480 - val_loss: 0.9206 - val_accuracy: 0.4636

Evaluatig Model--->

12/12 [==============================] - 3s 210ms/step - loss: 0.9339 - accuracy: 0.4358


In [ ]:
df2['E']= label_encoder.fit_transform(df2['E'])
y_E=to_categorical(df2['E'],dtype='int32')
# y_E.shape
model_E=buildmodel(X1,y_E)

Epoch 1/5
180/180 [==============================] - 66s 368ms/step - loss: 0.9530 - accuracy: 0.4532 - val_loss: 0.9513 - val_accuracy: 0.4606
Epoch 2/5
180/180 [==============================] - 67s 371ms/step - loss: 0.9484 - accuracy: 0.4565 - val_loss: 0.9534 - val_accuracy: 0.4606
Epoch 3/5
180/180 [==============================] - 65s 364ms/step - loss: 0.9469 - accuracy: 0.4572 - val_loss: 0.9520 - val_accuracy: 0.4413
Epoch 4/5
180/180 [==============================] - 66s 365ms/step - loss: 0.9483 - accuracy: 0.4550 - val_loss: 0.9586 - val_accuracy: 0.4413
Epoch 5/5
180/180 [==============================] - 66s 364ms/step - loss: 0.9465 - accuracy: 0.4524 - val_loss: 0.9539 - val_accuracy: 0.4413

Evaluatig Model--->

12/12 [==============================] - 3s 210ms/step - loss: 0.9697 - accuracy: 0.4465


In [ ]:
df2['A']= label_encoder.fit_transform(df2['A'])
y_A=to_categorical(df2['A'],dtype='int32')
model_A=buildmodel(X1,y_A)

Epoch 1/5
169/169 [==============================] - 72s 426ms/step - loss: 0.8516 - accuracy: 0.4793
Epoch 2/5
169/169 [==============================] - 72s 425ms/step - loss: 0.8324 - accuracy: 0.4981
Epoch 3/5
169/169 [==============================] - 72s 425ms/step - loss: 0.8023 - accuracy: 0.5123
Epoch 4/5
169/169 [==============================] - 72s 426ms/step - loss: 0.7847 - accuracy: 0.5691
Epoch 5/5
169/169 [==============================] - 71s 423ms/step - loss: 0.6888 - accuracy: 0.6586

Evaluatig Model--->

12/12 [==============================] - 3s 214ms/step - loss: 1.0054 - accuracy: 0.4947


In [ ]:
df2['N']= label_encoder.fit_transform(df2['N'])
y_N=to_categorical(df2['N'],dtype='int32')
model_N=buildmodel(X1,y_N)

Epoch 1/5
180/180 [==============================] - 77s 382ms/step - loss: 0.9819 - accuracy: 0.4200 - val_loss: 0.9812 - val_accuracy: 0.4428
Epoch 2/5
180/180 [==============================] - 66s 367ms/step - loss: 0.9816 - accuracy: 0.4335 - val_loss: 0.9907 - val_accuracy: 0.4354
Epoch 3/5
180/180 [==============================] - 66s 365ms/step - loss: 0.9822 - accuracy: 0.4491 - val_loss: 0.9794 - val_accuracy: 0.4428
Epoch 4/5
180/180 [==============================] - 66s 366ms/step - loss: 0.9828 - accuracy: 0.4401 - val_loss: 0.9829 - val_accuracy: 0.4428
Epoch 5/5
180/180 [==============================] - 66s 365ms/step - loss: 0.9816 - accuracy: 0.4375 - val_loss: 0.9806 - val_accuracy: 0.4428

Evaluatig Model--->

12/12 [==============================] - 3s 210ms/step - loss: 0.9579 - accuracy: 0.4866


In [ ]:
model_name =  'espejelomar/beto-base-cased'
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
spanish_model = TFBertModel.from_pretrained(model_name, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [ ]:
inputs=Input(shape=(100),name='input_text',dtype='int32')
bmodel=spanish_model.layers[0]
smodel=bmodel(inputs)[1]
dropout=Dropout(0.2)
output=dropout(smodel,training=False)
# entered=Dense(units=512)(output)
y=Dense(units=3)(output)
model=Model(inputs=inputs,outputs=y)

In [ ]:
optimizer = Adam(learning_rate=5e-05)
loss=CategoricalCrossentropy(from_logits=True)
accuracy=CategoricalAccuracy('accuracy')


In [ ]:
model.compile(optimizer=optimizer,loss=loss,metrics=[accuracy])

In [ ]:


model.save(os.path.join('model_A.h5', '/content/drive/MyDrive/Models/o_model.h5'))


NameError: ignored

In [ ]:
new_model = tf.keras.models.load_model(os.path.join(mname, pth))

In [ ]:
xt='''un extraño es libre son competitivos son mujeres'''

t=[xt,'sobre ti es cuando puedes hacer las cosas que quieras cuando quieras ayudarse mutuamente en situaciones difíciles apoyarse mutuamente en la vida cotidiana las cosas duran para siempre pero no se encuentran todo el tiempo la autoeficacia conduce al éxito en la vidasobre ti es cuando puedes hacer las cosas que quieras cuando quieras ayudarse mutuamente en situaciones difíciles apoyarse mutuamente en la vida cotidiana las cosas duran para siempre pero no se encuentran todo el tiempo la autoeficacia conduce al éxito en la vida']
x = tokenizer(
    text=t, 
    max_length=100,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    verbose = True)
p=new_model.predict(
  x=x["input_ids"]
)
p1=p.argmax(1)
p1
p2=label_encoder.inverse_transform(p1)
p2
p2=p2[0]
print(p2)

1


In [ ]:
tf.loads_model('')

In [ ]:
def predictions(xt, m_name):
  t=[xt,'sobre ti es cuando puedes hacer las cosas que quieras cuando quieras ayudarse mutuamente en situaciones difíciles apoyarse mutuamente en la vida cotidiana las cosas duran para siempre pero no se encuentran todo el tiempo la autoeficacia conduce al éxito en la vidasobre ti es cuando puedes hacer las cosas que quieras cuando quieras ayudarse mutuamente en situaciones difíciles apoyarse mutuamente en la vida cotidiana las cosas duran para siempre pero no se encuentran todo el tiempo la autoeficacia conduce al éxito en la vida']
  x = tokenizer(
      text=t, 
      max_length=100,
      truncation=True,
      padding=True, 
      return_tensors='tf',
      verbose = True)
  p=m_name.predict(
    x=x["input_ids"]
  )
  p1=p.argmax(1)
  p1
  p2=label_encoder.inverse_transform(p1)
  p2
  p2=p2[0]
  return p2


df2['Text'][12441]

NameError: ignored

In [ ]:
# x='''alguien que quiere ayudarte demasiado está lleno de amor y respeto los míos son terribles así que no sabría lo que es un buen hermano no tengo ninguno la vida está enamorada forma triste de vivir'''
# x='''un extraño es libre son competitivos son mujeres'''
x='''está diciendo que los vikingos mejor matan a los leones como si fueran stanford matando a los troyanos ayer  ppppppoker face pppoker face ¡la ciudad está encendida'''
O=predictions(x,model_O)
C=predictions(x,model_C)
E=predictions(x,model_E)
A=predictions(x,model_A)
N=predictions(x,model_N)
final = {'Text':x, 'O':O, 'C':C, 'E':E, 'A':A, 'N':N}
final                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

NameError: ignored